In [1]:
import transformers
transformers.__version__

'4.19.2'

In [2]:
#!pip install protobuf==3.20.*

In [3]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "mrm8488/mbart-large-finetuned-opus-es-en-translation"#mbart-large-finetuned-bible-es-en-translation"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [4]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


## _Test con los títulos de noticias_


In [5]:
import pandas as pd
df = pd.read_csv("dataset-loslagos-8months.csv", 
                 sep=";", 
                 engine='python')
df.columns = ["date", "media_outlet", "url", "title", "text"]
print(len(df))
df.head()

29598


,date,media_outlet,url,title,text
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At..."


In [6]:
from tqdm import tqdm
sub = df[:100].copy()
sub['sentiment'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - sentiment', total=sub.shape[0]):
    
    inputs = tokenizer(row['title'], return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)
    to_eng = tokenizer.decode(outputs[0])[4:-4]
    sentiment_value = nlp(to_eng)
    sub.at[index, "sentiment"] = sentiment_value[0]


sub rows - sentiment: 100%|███████████████████████████████████████████████████████████| 100/100 [03:28<00:00,  2.08s/it]


In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)
sub[['title', 'sentiment']]

,title,sentiment
0,Reconocen a guardaparques de la Región de Los Lagos como actores claves en la conservación,"{'label': 'POSITIVE', 'score': 0.9989738464355469}"
1,Con nuevos materiales comienza plan piloto en Saltos del Petrohué,"{'label': 'POSITIVE', 'score': 0.9602944254875183}"
2,Centro de Salud Familiar CESFAM Puerto Varas invita a prevenir el Cáncer Cervicouterino con extensión horaria para exámenes PAP,"{'label': 'NEGATIVE', 'score': 0.9650343060493469}"
3,Alcalde Tomás Gárate presidió por primera vez la octava sesión del Consejo Comunal de la Sociedad Civil COSOC Puerto Varas,"{'label': 'POSITIVE', 'score': 0.9859645366668701}"
4,Galería de Arte Machacoya realizará remate de obras de artistas de la zona,"{'label': 'POSITIVE', 'score': 0.9991785883903503}"
5,Municipio llamará a licitación construcción de pasarela peatonal en Los Notros,"{'label': 'NEGATIVE', 'score': 0.9589917659759521}"
6,3era Mesa de Reactivación Económica circunscribe su radio de acción,"{'label': 'NEGATIVE', 'score': 0.9504976868629456}"
7,Gremio médico rechaza cierre de camas críticas implementadas en Hospital Ancud,"{'label': 'NEGATIVE', 'score': 0.9135583639144897}"
8,Asistentes de educación municipal en Osorno verán incrementadas sus remuneraciones,"{'label': 'POSITIVE', 'score': 0.9925214648246765}"
9,CONADI llama a renovar directivas de comunidades indígenas,"{'label': 'NEGATIVE', 'score': 0.7058996558189392}"


In [8]:
sub['label'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - sentiment', total=sub.shape[0]):
    
    label = row['sentiment'].get('label')
    score = row['sentiment'].get('score')
    
    if(score > 0.9 and label=="NEGATIVE"): sub.at[index, "label"] = "negative"
    if(score > 0.8 and score < 0.9 and label=="NEGATIVE"): sub.at[index, "label"] = "neutral negative"
    if(score < 0.8): sub.at[index, "label"] = "neutral"
    if(score > 0.8 and score < 0.9 and label=="POSITIVE"): sub.at[index, "label"] = "neutral positive"
    if(score > 0.9 and label=="POSITIVE"): sub.at[index, "label"] = "positive"
        
    
    

sub rows - sentiment: 100%|█████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1060.84it/s]


In [9]:
sub.label.value_counts()

negative            47
positive            30
neutral             12
neutral negative     9
neutral positive     2
Name: label, dtype: int64

In [10]:
sub.loc[sub.label == 'neutral']

,date,media_outlet,url,title,text,sentiment,label
9,2021-10-01,seminariolocal,https://www.semanariolocal.cl/archivos/2198,CONADI llama a renovar directivas de comunidades indígenas,"En plena pandemia del Coronavirus, el 26 de junio de 2020, se promulgó la Ley 21.244 para prorrogar el mandato de los miembros de las directivas de comunidades y asociaciones indígenas, ello con el objeto de evitar procesos eleccionarios, asambleas y así proteger a los integrantes de las comunidades ante posibles contagios. Hoy, al finalizar el estado de excepción constitucional por catástrofe, las comunidades y asociaciones tendrán un plazo máximo de tres meses para realizar sus procesos eleccionarios, un hecho que debe ser considerado por los dirigentes, ya que las comunidades que después de tres meses no cuenten con su directiva actualizada no podrán recibir recursos públicos. Invitación “Con fecha 2 de julio del año 2020 se publicó la ley 21.244, y esta ley fue de Vital importancia para poder prorrogar la vigencia de las directivas de las comunidades y asociaciones indígenas que están registradas en CONADI. Esto permitió que, las comunidades y asociaciones durante la vigencia del Estado excepción constitucional, pudieran seguir postulando y participando de los fondos concursables de CONADI y otros servicios públicos”, explicó Ignacio Malig Meza, Director Nacional de CONADI. El directivo agregó que “como ha terminado el estado de excepción constitucional, ahora les corresponde por esta misma modificación legal realizar la elección de sus directivas dentro de los próximos tres meses. Así es que la invitación es a todas las comunidades y asociaciones cuya directiva haya expirado durante el estado de excepción constitucional a que ahora, dentro de los próximos tres meses, realicen las elecciones para poder seguir participando de CONADI y los demás servicios públicos que exigen que sus directivas se encuentren vigentes”. Actualización Para actualizar sus directivas, las comunidades y asociaciones indígenas deben realizar la elección de los integrantes de la directiva de acuerdo a la forma establecida en sus estatutos y luego deben concurrir con un acta firmada por todos los socios, para que sea registrada en cualquier oficina de CONADI. Esta acta, con la nueva directiva, también puede informar sobre socios fallecidos, incorporación de nuevos socios y renuncia de socios.","{'label': 'NEGATIVE', 'score': 0.7058996558189392}",neutral
11,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/regional/se-constituyo-nuevo-comite-para-la-gestion-del-riesgo-de-desastres-cogrid-12222,Se constituyó nuevo Comité para la Gestión del Riesgo de Desastres (COGRID),"En la tarde de ayer, se llevó a cabo el último COE regional y se realizó la primera sesión del nuevo Comité para la Gestión del Riesgo de Desastres denominado COGRID, el cual se encargará de la planificación y coordinación del Sistema a nivel regional, además de constituirse y ejercer las funciones descritas en la ley y el reglamento para abordar las fases del ciclo del riesgo de desastres. De la instancia pionera participó el Delegado Presidencial Regional de Los Lagos, Carlos Geisse, quien señaló “se actuará en coordinación con el gobernador regional, oyendo además a otras entidades y organismos públicos y privados, necesarios para abordar temas relevantes en la Gestión del Riesgo de Desastres”. El Gobernador Regional, Patricio Vallespín aseguró que “estamos iniciando el proceso de la nueva institucionalidad de emergencia, esperemos que lo convoquemos pocas veces, tenemos que estar para que en su minuto poder enfrentar ocurrencia de este tipo tengamos unidad de mando. Muy positiva reunión ha quedado claro van a ver instructivos que hay que generar, concordar tareas, creo que vamos a actuar muy coordinadamente”. Por su parte el director regional de la ONEMI manifestó la importancia de esta nueva instancia de coordinación regional, ya que “sigue los lineamientos de la nueva institucionalidad que dispone